In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls

Mounted at /content/drive
drive  sample_data


In [3]:
# move into project directory
repo_name = "crop-damage-classification"
%cd /content/drive/MyDrive/Personal-Projects/$repo_name
!ls

/content/drive/MyDrive/Personal-Projects/crop-damage-classification
common	     dataloading  Index.ipynb  output		     README.md	 visualization
config.yaml  experiments  index.py     preprocess	     run.yaml
data	     Index_bc.py  models       project-structure.md  transforms


In [4]:
# set up environment
# comment if not required
'''
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

# Following cells are for downloading data

In [4]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder
# comment if not required
!pip install boto3
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 13.1 MB/s eta 0:00:00


In [5]:
# setup some imports
#custom imports
from transforms.transforms import ToTensor, Resize, CenterCrop
#from dataloading.dataset import CropDataset
from common.utils import get_exp_params, init_config, get_config, save2config
from models.resnet18 import Resnet18
#from experiments.experiments import Experiment
from visualization.visualization import Visualization

#py imports
import random
import numpy as np
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [6]:
#Python file to define custom dataset for your project

from torch.utils.data import Dataset
import pandas as pd
import os
from skimage import io, transform
from common.utils import get_config
import torch

class CropDataset(Dataset):

    def __init__(self, label_csv_path, name2numlblmap, is_test = False, transforms = None):
        config = get_config()
        self.data_dir = config['data_dir']
        self.img_dir = config['img_dir']
        self.image_labels = pd.read_csv(os.path.join(self.data_dir, label_csv_path))
        self.transforms = transforms
        self.name2numlblmap = name2numlblmap
        self.is_test = is_test

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_tensor_path = os.path.join(self.img_dir, self.image_labels.loc[idx, 'filename'])
        img_tensor = io.imread(img_tensor_path)
        label = self.name2numlblmap[self.image_labels.loc[idx, 'damage']] if not(self.is_test) else ''
        sample = { 'id': self.image_labels.loc[idx, 'ID'],
            'image': img_tensor, 'label': label,
            'class': self.image_labels.loc[idx, 'damage'] if not(self.is_test) else '',
            'filename': self.image_labels.loc[idx, 'filename']
        }

        if self.transforms:
            sample = self.transforms(sample)

        return sample



In [26]:
from common.utils import get_exp_params
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from common.utils import get_accuracy, get_config, save_experiment_output
import pandas as pd
import os
import torch.nn.functional as F

class Experiment:

    def __get_optimizer(self, model, model_params, optimizer_name = 'Adam'):
        if optimizer_name == 'Adam':
            return torch.optim.Adam(model.parameters(), lr=model_params['lr'], weight_decay = model_params['weight_decay'], amsgrad = model_params['amsgrad'])
        elif optimizer_name == 'SGD':
            return torch.optim.SGD(model.parameters(), lr=model_params['lr'], weight_decay = model_params['weight_decay'], momentum = model_params['momentum'], nesterov= True)
        else:
            raise SystemExit("Error: no valid optimizer name passed! Check run.yaml file")


    def __init__(self, model, fr_train_dataset):
        self.exp_params = get_exp_params()
        self.model = model
        self.optimizer = self.__get_optimizer(self.model, self.exp_params['model'], self.exp_params['model']['optimizer'])
        self.fr_train_dataset = fr_train_dataset
        cfg = get_config()
        self.X_key = cfg['X_key']
        self.y_key = cfg['y_key']
        self.device = "cuda" if cfg['use_gpu'] else "cpu"

    def __loss_fn(self, loss_name = 'cross-entropy'):
        if loss_name == 'cross-entropy':
            return torch.nn.CrossEntropyLoss()
        else:
            raise SystemExit("Error: no valid loss function name passed! Check run.yaml")

    def __conduct_training(self, train_loader, val_loader):
        loss_fn = self.__loss_fn()
        tr_batch_num = len(train_loader)
        val_batch_num = len(val_loader)
        num_epochs = self.exp_params['train']['num_epochs']
        epoch_ivl = self.exp_params['train']['epoch_interval']
        batch_ivl = self.exp_params['train']['batch_interval']
        best_loss = 99999
        best_model = {}
        best_model_trlosshistory = []
        best_model_vallosshistory = []
        tr_loss_history = []
        val_loss_history = []
        tr_loss = 0.0
        for i in range(num_epochs):
            print(f'\tRunning Epoch {i}')
            self.model.train()
            running_loss = 0.0
            print(f'\t\tRunning through training dataset')
            for batch_idx, batch in enumerate(train_loader):
                self.optimizer.zero_grad()
                batch[self.X_key] = batch[self.X_key].float().to(self.device)
                batch[self.y_key] = batch[self.y_key].to(self.device)
                op = self.model(batch[self.X_key])
                loss = loss_fn(op, batch[self.y_key])
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
                if (batch_idx + 1) % batch_ivl == 0:
                    print(f'\t\tBatch {batch_idx + 1} Loss: {running_loss / (batch_idx + 1)}')
            tr_loss = running_loss / tr_batch_num
            tr_loss_history.append(tr_loss)

            print('\t\tRunning through validation set')
            self.model.eval()
            val_loss = 0.0
            val_acc = 0
            for batch_idx, batch in enumerate(val_loader):
                batch[self.X_key] = batch[self.X_key].float().to(self.device)
                batch[self.y_key] = batch[self.y_key].to(self.device)
                lop = self.model(batch[self.X_key])
                loss = loss_fn(lop, batch[self.y_key])
                lop_lbls = torch.argmax(lop, 1)
                loss.backward()
                val_loss += loss.item()
                val_acc += get_accuracy(lop_lbls, batch[self.y_key])

                if (batch_idx + 1) % batch_ivl == 0:
                    print(f'\t\tBatch {batch_idx + 1} Last Model Loss: {val_loss / (batch_idx + 1)}')
                    print(f'\t\tBatch {batch_idx + 1} Best Model Loss: {val_loss / (batch_idx + 1)}')
            val_loss /= val_batch_num
            val_acc /= val_batch_num
            val_loss_history.append(val_loss)
            if val_loss < best_loss:
                best_loss = val_loss
                best_model = self.model
                best_acc = val_acc
                best_model_trlosshistory = tr_loss_history
                best_model_vallosshistory = val_loss_history
            if (i+1) % epoch_ivl == 0:
                print(f'Epoch {i} Training Loss: {tr_loss}')
                print(f"Epoch {i} Validation Loss: {val_loss}")
                print(f"Epoch {i} Validation Accuracy: {val_acc}\n")


        model_info = {
            'best_model': best_model if best_model != {} else {},
            'best_model_valloss': best_loss,
            'best_model_valacc': best_acc,
            'best_model_trlosshistory': torch.tensor(best_model_trlosshistory),
            'best_model_vallosshistory': torch.tensor(best_model_vallosshistory),
            'last_model': self.model,
            'last_model_valloss': val_loss,
            'last_model_valacc': val_acc,
            'last_model_trlosshistory': torch.tensor(tr_loss_history),
            'last_model_vallosshistory': torch.tensor(val_loss_history)
        }

        return model_info


    def train(self):
        train_loader = {}
        val_loader = {}
        self.model = self.model.to(self.device)
        if self.exp_params['train']['val_split_method'] == 'k-fold':
            k = self.exp_params['train']['k']
            vp = self.exp_params['train']['val_percentage']
            fl = len(self.fr_train_dataset)
            fr = list(range(fl))
            vlen = int(vp * fl)
            vset_len = fl // k
            val_eei = list(range(vset_len, fl, vset_len))
            si = 0
            bestm_acc = 0.0
            lastm_acc = 0.0
            lastm_loss = 0.0
            bestm_loss = 0.0
            bestm_tlh = torch.zeros(self.exp_params['train']['num_epochs'])
            bestm_vlh = torch.zeros(self.exp_params['train']['num_epochs'])
            lastm_tlh = torch.zeros(self.exp_params['train']['num_epochs'])
            lastm_vlh = torch.zeros(self.exp_params['train']['num_epochs'])

            for vi, ei in enumerate(val_eei):
                print(f"Running split {vi}")
                val_idxs = fr[si:ei]
                tr_idxs = fr[ei:]
                si = ei
                train_dataset = Subset(self.fr_train_dataset, tr_idxs)
                val_dataset = Subset(self.fr_train_dataset, val_idxs)

                train_loader = DataLoader(train_dataset,
                batch_size = self.exp_params['train']['batch_size'],
                shuffle = self.exp_params['train']['shuffle_data']
                )
                val_loader = DataLoader(val_dataset,
                    batch_size = self.exp_params['train']['batch_size'],
                    shuffle = self.exp_params['train']['shuffle_data']
                )
                model_info = self.__conduct_training(train_loader, val_loader)
                bestm_acc += model_info['best_model_valacc']
                bestm_loss += model_info['best_model_valloss']
                bestm_tlh += model_info['best_model_trlosshistory']
                bestm_vlh += model_info['best_model_vallosshistory']
                lastm_acc += model_info['last_model_valacc']
                lastm_loss += model_info['last_model_valloss']
                lastm_tlh += model_info['last_model_trlosshistory']
                lastm_vlh += model_info['last_model_vallosshistory']
            bestm_loss/=k
            bestm_acc/=k
            bestm_tlh/=k
            bestm_vlh/=k
            lastm_loss/=k
            lastm_acc/=k
            lastm_tlh/=k
            lastm_vlh/=k
            model_info['best_model_valacc'] = bestm_acc
            model_info['best_model_valloss'] = bestm_loss
            model_info['best_model_trlosshistory'] = bestm_tlh
            model_info['best_model_vallosshistory'] = bestm_vlh
            model_info['last_model_valacc'] = lastm_acc
            model_info['last_model_valloss'] = lastm_loss
            model_info['last_model_trlosshistory'] = lastm_tlh
            model_info['last_model_vallosshistory'] = lastm_vlh
            return model_info
        elif self.exp_params['train']['val_split_method'] == 'fix-split':
            print("Running straight split")
            vp = self.exp_params['train']['val_percentage']
            vlen = int(vp * len(self.fr_train_dataset))
            val_idxs = np.random.randint(0, len(self.fr_train_dataset), vlen).tolist()
            tr_idxs = [idx not in val_idxs for idx in range(len(self.fr_train_dataset))]
            train_dataset = Subset(self.fr_train_dataset, tr_idxs)
            val_dataset = Subset(self.fr_train_dataset, val_idxs)

            train_loader = DataLoader(train_dataset,
                batch_size = self.exp_params['train']['batch_size'],
                shuffle = self.exp_params['train']['shuffle_data']
            )
            val_loader = DataLoader(val_dataset,
                batch_size = self.exp_params['train']['batch_size'],
                shuffle = self.exp_params['train']['shuffle_data']
            )
            model_info = self.__conduct_training(train_loader, val_loader)
            return model_info
        else:
            raise SystemExit("Error: no valid split method passed! Check run.yaml")

    def save_model(self, model, chkpt_info, model_type, is_chkpt = True, is_best = True):
        save_experiment_output(model, chkpt_info, self.exp_params,
            is_chkpt, model_type, is_best)

    def test(self, model, test_dataset, lbl_dict):
        model = model.to(self.device)
        test_loader = DataLoader(test_dataset, batch_size = self.exp_params["train"]["batch_size"], shuffle = True)
        model.eval()
        loss_fn = self.__loss_fn(self.exp_params["train"]["loss"])
        running_loss = 0.0
        acc = 0
        num2class = lambda x: lbl_dict[x.item()]
        sub_lbls = ['ID', 'DR', 'G', 'ND', 'WD', 'other']
        results_df = pd.DataFrame([], columns = sub_lbls)
        print("Running through test dataset")
        with torch.no_grad():
            for bi, batch in enumerate(test_loader):
                print(f"\tRunning through batch {bi}")
                batch[self.X_key] = batch[self.X_key].float().to(self.device)
                op = F.softmax(model(batch[self.X_key].float()), 1)
                if self.device == "cuda":
                    batch[self.X_key] = batch[self.X_key].to("cpu")
                else:
                    del batch[self.X_key]
                # predicted labels
                '''
                oplbls = torch.argmax(op, 1)
                classlbls = list(map(num2class, oplbls))
                '''
                res = [[id] + preds for id,preds in zip(batch['id'], op.tolist())]
                batch_df = pd.DataFrame(res, columns = sub_lbls)
                results_df = pd.concat([results_df, batch_df], 0)
        results_df.to_csv(os.path.join(self.output_dir, "results.csv"), index = False)



In [8]:
import boto3
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config
from tqdm.notebook import tqdm

def get_file_folders(s3_client, bucket_name, prefix=""):
    file_names = []
    folders = []

    default_kwargs = {
        "Bucket": bucket_name,
        "Prefix": prefix
    }
    next_token = ""

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        response = s3_client.list_objects_v2(**updated_kwargs)
        contents = response.get("Contents")

        for result in contents:
            key = result.get("Key")
            if key[-1] == "/":
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get("NextContinuationToken")

    return file_names, folders

def download_files(s3_client, bucket_name, local_path, file_names, folders):
    local_path = Path(local_path)

    for folder in tqdm(folders):
        folder_path = Path.joinpath(local_path, folder)
				# Create all folders in the path
        folder_path.mkdir(parents=True, exist_ok=True)

    for file_name in tqdm(file_names):
        file_path = Path.joinpath(local_path, file_name)
				# Create folder for parent directory
        file_path.parent.mkdir(parents=True, exist_ok=True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )

data_path = 'data/input/images'
if not(os.path.exists(os.path.join(os.getcwd(), data_path))):
    client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    file_names, folders = get_file_folders(client, 'cgiar-crop-damage-classification-challenge')
    download_files(
        client,
        'cgiar-crop-damage-classification-challenge',
        "/content/drive/MyDrive/Personal-Projects/crop-damage-classification/data/input",
        file_names,
        folders
    )

In [8]:
# initialize directories and config data
init_config()
config = get_config()
print('Config parameters\n')
print(config)

Config parameters

{'X_key': 'image', 'data_dir': '/content/drive/MyDrive/Personal-Projects/crop-damage-classification/data', 'img_dir': '/content/drive/MyDrive/Personal-Projects/crop-damage-classification/data/input/images', 'root_dir': '/content/drive/MyDrive/Personal-Projects/crop-damage-classification', 'use_gpu': True, 'y_key': 'label'}


In [9]:
# read experiment parameters
exp_params = get_exp_params()
print('Experiment parameters\n')
print(exp_params)

Experiment parameters

{'transform': {'resize_dim': 256, 'crop_dim': 224}, 'train': {'shuffle_data': True, 'batch_size': 128, 'val_split_method': 'k-fold', 'k': 2, 'val_percentage': 20, 'loss': 'cross-entropy', 'batch_interval': 512, 'epoch_interval': 1, 'num_epochs': 1}, 'model': {'name': 'resnet18', 'optimizer': 'Adam', 'lr': 0.001, 'weight_decay': 1e-05, 'amsgrad': False, 'momentum': 0.9}, 'test_model': False}


In [10]:
#initialize randomness seed
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [11]:
#preprocess data or load preprocessed data

#build label dict
label_dict = {
    'DR': 0,
    'G': 1,
    'ND': 2,
    'WD': 3,
    'other': 4
}

class_dict = {
    0: 'DR',
    1: 'G',
    2: 'ND',
    3: 'WD',
    4: 'other'
}

In [12]:
#save X_key and y_key
save2config('X_key', 'image')
save2config('y_key', 'label')

#transform data
data_transforms = transforms.Compose([ToTensor(), Resize(exp_params['transform']['resize_dim']), CenterCrop(exp_params['transform']['crop_dim'])])

#convert to dataset
ftr_dataset = CropDataset('input/Train.csv', label_dict, False, transforms=data_transforms)
test_dataset = CropDataset('input/Test.csv', label_dict, True, transforms=data_transforms)
smlen = int(0.05 * len(ftr_dataset))
smftr_dataset = torch.utils.data.Subset(ftr_dataset, list(range(smlen)))
print('Full train dataset length:', len(ftr_dataset))
print('Test dataset length:', len(test_dataset))
print('Subset train dataset length:', smlen, '\n')



Full train dataset length: 26068
Test dataset length: 8663
Subset train dataset length: 1303 



In [13]:
#model import

if exp_params['model']['name'] == 'resnet18':
    model = Resnet18(5, False)
else:
    raise SystemExit("Error: Invalid model name passed! Check run.yaml")


In [ ]:
#running experiment on small subset of the dataset
exp = Experiment(model, smftr_dataset)
model_info = exp.train()
print("\nModel validation results")

#visualization results
vis = Visualization(model_info)
vis.get_results()

Running split 0
	Running Epoch 0
		Running through training dataset


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


		Running through validation set


In [ ]:
'''
#model training on full dataset
exp = Experiment(model, ftr_dataset)
model_info = exp.train()
print("\nModel validation results")

#visualization results
vis = Visualization(model_info)
vis.get_results()
'''

In [27]:
#running experiment on small subset of the dataset
exp = Experiment(model, smftr_dataset)

In [ ]:
#model testing
print("Testing Best Model")
exp.test(model, test_dataset, class_dict)
#print("\nTesting Last Model")
#exp.test(model_info["last_model"], test_dataset, label_dict)

Testing Best Model
Running through test dataset


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


	Running through batch 0


<ipython-input-26-9ce107c84404>:230: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


	Running through batch 1


...

<ipython-input-26-9ce107c84404>:230: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


	Running through batch 63


<ipython-input-26-9ce107c84404>:230: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


	Running through batch 64


<ipython-input-26-9ce107c84404>:230: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
